In [0]:
import sys, os, shutil
from glob import glob
import numpy as np

## Change Runtime on Colab

## Get Data

In [0]:
!apt-get install tree
!pip install tqdm
!pip install -U kaggle-cli

In [0]:
def chdir_home():
    pwd = !pwd
    if pwd[0].count('/') > 1 :
        os.chdir((pwd[0].count('/')-1)*'../')

In [7]:
# Download in home directory
chdir_home()
!kg download -u skakki -p akshaykumar -c dogs-vs-cats-redux-kernels-edition

downloading https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/test.zip

test.zip 100% |#####################################| Time: 0:00:06  45.0 MiB/s

downloading https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/train.zip

train.zip 100% |####################################| Time: 0:00:12  44.2 MiB/s

downloading https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/sample_submission.csv

sample_submission.csv 100% |########################| Time: 0:00:00 479.8 KiB/s



In [0]:
!mkdir -p data/redux
!unzip train.zip -d data/redux
!unzip test.zip -d data/redux

In [0]:
!mkdir -p data/redux/test/unknown/
!mkdir -p data/redux/train/cats
!mkdir -p data/redux/train/dogs
!mkdir -p data/redux/valid/cats
!mkdir -p data/redux/valid/dogs
!mkdir -p data/redux/results

!mkdir -p data/redux/sample/train/cats
!mkdir -p data/redux/sample/train/dogs
!mkdir -p data/redux/sample/valid/cats
!mkdir -p data/redux/sample/valid/dogs
!mkdir -p data/redux/sample/test/unknown
!mkdir -p data/redux/sample/results

In [0]:
# these paths are when you are at home directory
chdir_home()

path = 'data/redux/'
# path = 'data/redux/sample/'

train_path = path+'train/'
val_path = path+'valid/'
test_path = path+'test/'
sample_path = path+'sample/'
results_path = path+'results/'
model_path = path+'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [0]:
# copy 1200 images from train and 600 from test to sample
g = glob(train_path+'*.jpg')
g = np.random.permutation(g)
for i in range(1200):
    shutil.copy(g[i], sample_path+'train/')

g = glob(test_path+'*.jpg')
g = np.random.permutation(g)
for i in range(600):
    shutil.copy(g[i], sample_path+'test/')


# Move 100 images from sample/train to sample/valid
g = glob(sample_path+'train/*.jpg')
g = np.random.permutation(g)
for i in range(100):
    shutil.move(g[i], sample_path+'valid/')


# move 2000 images from train to valid directory

g = glob(train_path+'*.jpg')
g = np.random.permutation(g)
for i in range(2000):
    shutil.move(g[i], val_path)



In [0]:
# move image in corresponsing label directories

!mv sample_submission.csv data/redux/

!mv data/redux/test/*.jpg data/redux/test/unknown/
!mv data/redux/train/cat*.jpg data/redux/train/cats/
!mv data/redux/train/dog*.jpg data/redux/train/dogs/
!mv data/redux/valid/dog*.jpg data/redux/valid/dogs/
!mv data/redux/valid/cat*.jpg data/redux/valid/cats/

!mv data/redux/sample/valid/cat*.jpg data/redux/sample/valid/cats/
!mv data/redux/sample/valid/dog*.jpg data/redux/sample/valid/dogs/
!mv data/redux/sample/train/cat*.jpg data/redux/sample/train/cats/
!mv data/redux/sample/train/dog*.jpg data/redux/sample/train/dogs/
!mv data/redux/sample/test/*.jpg data/redux/sample/test/unknown/


In [0]:
!tree data/ -d

## Install Packages

Following packages need to installed everytime. Also first change directory to the location where this notebook is located. 

In [0]:
!pip install tqdm
!pip install bcolz
!pip install keras==1.2

## Using inbuilt finetune

In [26]:
chdir_home()
os.chdir('drive/workspace/Dogs-Cats-Redux/')

%matplotlib inline
import sys, os, shutil
from glob import glob
import numpy as np
from utils import *
from vgg16 import Vgg16
from PIL import Image
from keras.preprocessing import image
from sklearn.metrics import confusion_matrix
from IPython.display import FileLink

from keras.utils.np_utils import to_categorical

chdir_home()

Using TensorFlow backend.


In [0]:
batch_size=64
no_epochs=1

path = 'data/redux/'
# path = 'data/redux/sample/'

train_path = path+'train/'
val_path = path+'valid/'
test_path = path+'test/'
results_path = path+'results/'

In [33]:
vgg = Vgg16()
train_batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(val_path, batch_size=batch_size)
vgg.finetune(train_batches)
vgg.model.optimizer.lr = 0.01

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [0]:
vgg.fit(train_batches, val_batches, no_epochs)

Epoch 1/1
22976/23000 [============================>.] - ETA: 0s - loss: 1.4647 - acc: 0.8758